# Importing

In [ ]:
# # keep runtime
# function ClickConnect(){
# console.log("Working"); 
# document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click();
# }
# var clicker = setInterval(ClickConnect,60000);
# # stop keeping runtime 
# clearInterval(clicker);

from google.colab import drive
drive.mount('/content/drive')

!pip install --upgrade scikit-learn
!pip install --upgrade statsmodels
!pip install --upgrade xgboost

# # lightgbm gpu version
# !pip uninstall lightgbmt --yes
# !pip install --upgrade cmake
# !git clone --recursive https://github.com/Microsoft/LightGBM
# %cd LightGBM
# !mkdir build
# %cd build
# !cmake -DUSE_GPU=1 ..
# !make -j$(nproc)
# %cd ../python-package
# !python3 setup.py install --gpu

!pip install --upgrade catboost
!pip install --upgrade tensorflow
!pip install --upgrade keras
!pip install --upgrade keras-tuner
!pip install --upgrade tensorflow-addons
!pip install --upgrade optuna
!pip install --upgrade wandb

import os
import sys
sys.path.append("/content/drive/MyDrive/Colab Notebooks/projects/DA_Platform")
from DA_v5 import *
import wandb
from wandb.keras import WandbCallback
from optuna.integration.wandb import WeightsAndBiasesCallback as optuna_wnbcallback
from optuna import pruners
import shutil
from glob import glob
from IPython.display import Image, display

import sys
from multiprocessing import cpu_count
import copy
import pickle
import warnings
from datetime import datetime, timedelta
from time import time, sleep, mktime
from matplotlib import font_manager as fm, rc, rcParams
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import re
import random
import wandb
from wandb.keras import WandbCallback
import shutil

import numpy as np
from numpy import array, nan, random as rnd, where
import pandas as pd
from pandas import DataFrame as dataframe, Series as series, isna, read_csv
from pandas.tseries.offsets import DateOffset
import statsmodels.api as sm
from scipy.stats import f_oneway

from sklearn import preprocessing as prep
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split as tts, GridSearchCV as GridTuner, StratifiedKFold, KFold
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler, RobustScaler
from sklearn import metrics
from sklearn.pipeline import make_pipeline

from sklearn import linear_model as lm
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as qda
from sklearn import svm
import lightgbm as lgb
import xgboost as xgb
import catboost as cat
from sklearn import neighbors as knn
from sklearn import ensemble
from optuna import distributions as optuna_dist, visualization as optuna_plt, Trial, create_study
from optuna.integration import OptunaSearchCV
from optuna.samplers import TPESampler
from optuna.logging import set_verbosity as optuna_set_verbose

# ===== tensorflow =====
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras import activations
from tensorflow.keras import optimizers
from tensorflow.keras import metrics as tf_metrics
from tensorflow.keras import callbacks as tf_callbacks
from tqdm.keras import TqdmCallback
import tensorflow_addons as tfa
import keras_tuner as kt
from keras_tuner import HyperModel
from tensorflow.keras.utils import plot_model

# ===== timeseries =====
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from sklearn.model_selection import TimeSeriesSplit
from tensorflow.keras.preprocessing import timeseries_dataset_from_array as make_ts_tensor

# global setting
warnings.filterwarnings(action='ignore')
rcParams['axes.unicode_minus'] = False
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', 1000)
pd.set_option('max_colwidth', 200)

rcParams['axes.unicode_minus'] = False
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
os.environ["WANDB_API_KEY"] = "6f810b088fcc6b9eaaa56c1e52cfd37836606240"
folder_path = "/content/drive/MyDrive/Colab Notebooks/projects/Dacon/job_recommendation/"

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    tf.config.experimental.set_memory_growth(gpus[0], True)
  except RuntimeError as e:
    print(e)

# Preprocessing & Feature engineering

In [ ]:
# ===== Preprocessing =====
target_var = "target"
full_x = read_csv(folder_path + "train.csv", encoding="utf-8")
full_x.drop("id", axis=1, inplace=True); full_x.drop_duplicates(inplace=True)
full_y = full_x[target_var]; full_x.drop(["person_rn", "contents_rn", "target"], axis=1, inplace=True)
test_x = read_csv(folder_path + "test.csv", encoding="utf-8")
test_x.drop(["id", "person_rn", "contents_rn"], axis=1, inplace=True)

In [ ]:
full_y.shape

(501951,)

In [ ]:
# ===== Preprocessing =====
target_var = "target"
full_x = read_csv(folder_path + "train.csv", encoding="utf-8")
full_x.drop("id", axis=1, inplace=True); full_x.drop_duplicates(inplace=True)
full_y = full_x[target_var]; full_x.drop(["person_rn", "contents_rn", "target"], axis=1, inplace=True)
test_x = read_csv(folder_path + "test.csv", encoding="utf-8")
test_x.drop(["id", "person_rn", "contents_rn"], axis=1, inplace=True)

print(full_x.isna().sum().sum())
print(test_x.isna().sum().sum())

full_x.info()
full_x.head(10)
full_x["contents_open_dt"] = pd.to_datetime(full_x["contents_open_dt"])

test_x.info()
test_x.head(10)
test_x["contents_open_dt"] = pd.to_datetime(test_x["contents_open_dt"])

date_vars = ["contents_open_dt"]

# Feature Engineering
# 1. 일자 관련
# train set
full_x["quarter"] = full_x["contents_open_dt"].dt.quarter
full_x["month"] = full_x["contents_open_dt"].dt.month
full_x["season"] = full_x["contents_open_dt"].apply(getSeason)
full_x["week_of_month"] = full_x["contents_open_dt"].apply(week_of_month)
# monday 0, sunday 6
full_x["weekday"] = full_x["contents_open_dt"].dt.weekday
full_x["hour"] = full_x["contents_open_dt"].dt.hour
full_x["office_hour"] = [1 if 18 >= i >= 9 else 0 for i in full_x["contents_open_dt"].dt.hour]

# sin, cos 주기성 변환 파생변수 컬럼추가
time_zero = datetime(1970, 1, 1, 0, 0, 0)
day_to_sec = 24 * 60 * 60
year_to_sec = 365.2425 * day_to_sec
full_x["freq_sin_year"] = np.sin(((full_x["contents_open_dt"] - time_zero).dt.total_seconds() / year_to_sec) * 2 * np.pi)
full_x["freq_cos_year"] = np.cos(((full_x["contents_open_dt"] - time_zero).dt.total_seconds() / year_to_sec) * 2 * np.pi)
full_x["freq_sin_day"] = np.sin(((full_x["contents_open_dt"] - time_zero).dt.total_seconds() / day_to_sec) * 2 * np.pi)
full_x["freq_cos_day"] = np.cos(((full_x["contents_open_dt"] - time_zero).dt.total_seconds() / day_to_sec) * 2 * np.pi)

# bool to int
full_x[["d_l_match_yn", "d_m_match_yn", "d_s_match_yn", "h_l_match_yn", "h_m_match_yn", "h_s_match_yn"]] = \
    full_x[["d_l_match_yn", "d_m_match_yn", "d_s_match_yn", "h_l_match_yn", "h_m_match_yn", "h_s_match_yn"]].astype("int")

# test set
test_x["quarter"] = test_x["contents_open_dt"].dt.quarter
test_x["month"] = test_x["contents_open_dt"].dt.month
test_x["season"] = test_x["contents_open_dt"].apply(getSeason)
test_x["week_of_month"] = test_x["contents_open_dt"].apply(week_of_month)
# monday 0, sunday 6
test_x["weekday"] = test_x["contents_open_dt"].dt.weekday
test_x["hour"] = test_x["contents_open_dt"].dt.hour
test_x["office_hour"] = [1 if 18 >= i >= 9 else 0 for i in test_x["contents_open_dt"].dt.hour]

# sin, cos 주기성 변환 파생변수 컬럼추가
time_zero = datetime(1970, 1, 1, 0, 0, 0)
day_to_sec = 24 * 60 * 60
year_to_sec = 365.2425 * day_to_sec
test_x["freq_sin_year"] = np.sin(((test_x["contents_open_dt"] - time_zero).dt.total_seconds() / year_to_sec) * 2 * np.pi)
test_x["freq_cos_year"] = np.cos(((test_x["contents_open_dt"] - time_zero).dt.total_seconds() / year_to_sec) * 2 * np.pi)
test_x["freq_sin_day"] = np.sin(((test_x["contents_open_dt"] - time_zero).dt.total_seconds() / day_to_sec) * 2 * np.pi)
test_x["freq_cos_day"] = np.cos(((test_x["contents_open_dt"] - time_zero).dt.total_seconds() / day_to_sec) * 2 * np.pi)

# bool to int
test_x[["d_l_match_yn", "d_m_match_yn", "d_s_match_yn", "h_l_match_yn", "h_m_match_yn", "h_s_match_yn"]] = \
test_x[["d_l_match_yn", "d_m_match_yn", "d_s_match_yn", "h_l_match_yn", "h_m_match_yn", "h_s_match_yn"]].astype("int")

# data check
full_x.info()
full_x.head(10)

test_x.info()
test_x.head(10)

# distribution check
for i in full_x:
    print(i)
    print(full_x[i].value_counts().sort_index(), "\n")
    print(test_x[i].value_counts().sort_index(), "\n")

bin_vars = ["d_l_match_yn", "d_m_match_yn", "d_s_match_yn", "h_l_match_yn", "h_m_match_yn", "h_s_match_yn", "person_attribute_a",
            "contents_attribute_j", "contents_attribute_k", "office_hour"]
cat_vars = ["person_prefer_c", "contents_attribute_i", "contents_attribute_a", "contents_attribute_j_1", "contents_attribute_c",
            "contents_attribute_m", "quarter", "month", "season", "week_of_month", "weekday", "hour"]
ord_vars = ["person_attribute_a_1", "person_attribute_b", "person_prefer_e", "contents_attribute_e"]


# <categorical feature engineering>
# check categorical compatibility
for i in bin_vars + cat_vars:
    print(full_x[i].value_counts().sort_index(), "\n")
    # print(test_x[i].value_counts().sort_index(), "\n")
    if len(full_x[i].value_counts()) != len(test_x[i].value_counts()):
        print(i, "---> length of category is not matched")

# # visualization
# for i in bin_vars + cat_vars:
#     createFolder(folder_path + "graphs")
#     fig, ax = plt.subplots(figsize=(10, 6))
#     graph = sns.countplot(x=full_x[i], hue=full_y)
#     change_width(ax, 0.2)
#     plt.title(i, fontsize=15, fontweight="bold", pad=15)
#     plt.savefig(folder_path + "graphs/" + i + ".png", dpi=300)
#     plt.close()
#     # break

# office_hour (16 ~ 8 based on hour feature graph)
full_x["office_hour"] = [1 if 16 >= i >= 8 else 0 for i in full_x["hour"]]
test_x["office_hour"] = [1 if 16 >= i >= 8 else 0 for i in test_x["hour"]]
# month_4to9 (16 ~ 8 based on month feature graph)
full_x["month_4to9"] = [1 if 9 >= i >= 4 else 0 for i in full_x["month"]]
test_x["month_4to9"] = [1 if 9 >= i >= 4 else 0 for i in test_x["month"]]

# # drop the features only valid on train set
# full_x.drop(["quarter", "month", "season", "hour"], axis=1, inplace=True)
# test_x.drop(["quarter", "month", "season", "hour"], axis=1, inplace=True)
# bin_vars.append("month_4to9")
# for i in ["quarter", "month", "season", "hour"]: cat_vars.remove(i)

# drop the features only valid on train set
full_x.drop(["month", "season", "hour"], axis=1, inplace=True)
test_x.drop(["month", "season", "hour"], axis=1, inplace=True)
bin_vars.append("month_4to9")
for i in ["month", "season", "hour"]: cat_vars.remove(i)

# check categorical compatibility
for i in bin_vars + cat_vars:
    # print(full_x[i].value_counts().sort_index(), "\n")
    # print(test_x[i].value_counts().sort_index(), "\n")
    if len(full_x[i].value_counts()) != len(test_x[i].value_counts()):
        print(i, "---> length of category is not matched")

# # visualization
# for i in bin_vars + cat_vars:
#     createFolder(folder_path + "graphs")
#     fig, ax = plt.subplots(figsize=(10, 6))
#     graph = sns.countplot(x=full_x[i], hue=full_y)
#     change_width(ax, 0.2)
#     plt.title(i, fontsize=15, fontweight="bold", pad=15)
#     plt.savefig(folder_path + "graphs/" + i + ".png", dpi=300)
#     plt.close()
#     # break


# # <numerical feature engineering>
num_vars = diff(full_x.columns, bin_vars + cat_vars + ord_vars + date_vars)


# # visualization
# for i in num_vars:
#     createFolder(folder_path + "graphs")
#     fig, ax = plt.subplots(figsize=(10, 6))
#     graph = sns.boxenplot(x=full_y, y=full_x[i], palette=sns.hls_palette())
#     graph.set_title(i, fontsize=13, fontweight="bold", pad=15)
#     graph.set_xlabel(graph.get_xlabel(), fontsize=12, fontweight="bold", labelpad=15)
#     graph.set_ylabel(graph.get_ylabel(), fontsize=12, fontweight="bold", labelpad=15)
#     plt.savefig(folder_path + "graphs/" + i + ".png", dpi=300)
#     plt.close()
#     # break

# # drop the features which has only unique value
# full_x.drop(["person_prefer_f", "person_prefer_g", "freq_sin_year", "freq_sin_day"], axis=1, inplace=True)
# test_x.drop(["person_prefer_f", "person_prefer_g", "freq_sin_year", "freq_sin_day"], axis=1, inplace=True)
# for i in ["person_prefer_f", "person_prefer_g", "freq_sin_year", "freq_sin_day"]: num_vars.remove(i)

# drop the features which has only unique value
full_x.drop(["person_prefer_f", "person_prefer_g",], axis=1, inplace=True)
test_x.drop(["person_prefer_f", "person_prefer_g",], axis=1, inplace=True)
for i in ["person_prefer_f", "person_prefer_g"]: num_vars.remove(i)

# matching number of columns check
print(len(num_vars) + len(bin_vars) + len(cat_vars) + len(date_vars) == len(full_x.columns))

# data check
full_x.info()
full_x.head(10)

test_x.info()
test_x.head(10)

label_encoder = MyLabelEncoder()
full_x = label_encoder.fit_transform(full_x, bin_vars + cat_vars + ord_vars)
test_x = label_encoder.transform(test_x)

oh_encoder = MyOneHotEncoder(label_encoder.dic_cat)
full_x_oh = oh_encoder.fit_transform(full_x, cat_vars)
test_x_oh = oh_encoder.transform(test_x)

full_x_oh.info()
full_x_oh.head(10)
test_x_oh.info()
test_x_oh.head(10)

full_x.isna().sum().sum()
test_x.isna().sum().sum()
full_x_oh.isna().sum().sum()
test_x_oh.isna().sum().sum()

# drop date columns
full_x.drop(date_vars, axis=1, inplace=True)
full_x_oh.drop(date_vars, axis=1, inplace=True)
test_x.drop(date_vars, axis=1, inplace=True)
test_x_oh.drop(date_vars, axis=1, inplace=True)

# Modeling

In [ ]:
# ===== modeling =====
# <10-folds inference>

# fold spliter setting
kfolds_spliter = StratifiedKFold(10, random_state=42, shuffle=True)
categoIdx = findIdx(full_x.columns, cat_vars)
cut_off = 0.5

# result container setting
val_prob = np.zeros((full_x.shape[0], 2))
test_prob = np.zeros((test_x.shape[0], 2))
val_perf = []

# optuna function
def optuna_objective_function(trial: Trial, train_x, train_y, val_x, val_y, model_name, ntrees=None, eta=None):
    if model_name == "CAT_GBM":
        tuning_params = {
            # "n_estimators" : trial.suggest_int("n_estimators", 500, 5000, step=500),
            # "learning_rate" : trial.suggest_categorical("learning_rate", [1e-3, 5e-3, 1e-2, 5e-2, 1e-1]),
            "max_depth": trial.suggest_int("max_depth", 4, 8, step=1),
            "bagging_temperature": trial.suggest_float("bagging_temperature", 0.05, 0.95, step=0.05),
            # "rsm": trial.suggest_float("rsm", 0.5, 0.9, step=0.05),
            "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 0.01, 10.0, step=0.01),
            "random_strength": trial.suggest_float("random_strength", 0.01, 1.5, step=0.01),
            "min_child_samples": trial.suggest_int("min_child_samples", 3, 299, step=2),  
            "scale_pos_weight": trial.suggest_float("scale_pos_weight", 1.0, 3.0, step=0.01)
        }

        model = cat.CatBoostClassifier(boosting_type="Plain", loss_function="Logloss", task_type="GPU",
                                    n_estimators=int(ntrees * 0.2),
                                    learning_rate=eta / 10,
                                    one_hot_max_size=3, leaf_estimation_method="Gradient",
                                    # leaf_estimation_iterations=5,
                                    # max_ctr_complexity=2,
                                    logging_level="Silent", random_state=fold, thread_count=cpu_count(),
                                    **tuning_params)
        model.fit(train_x, train_y, cat_features=categoIdx)
    elif model_name == "CAT_ORD":
        tuning_params = {
            # "n_estimators" : trial.suggest_int("n_estimators", 500, 5000, step=500),
            # "learning_rate" : trial.suggest_categorical("learning_rate", [1e-3, 5e-3, 1e-2, 5e-2, 1e-1]),
            "max_depth": trial.suggest_int("max_depth", 4, 8, step=1),
            "bagging_temperature": trial.suggest_float("bagging_temperature", 0.05, 0.95, step=0.05),
            # "rsm": trial.suggest_float("rsm", 0.5, 0.9, step=0.05),
            "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 0.01, 10.0, step=0.01),
            "random_strength": trial.suggest_float("random_strength", 0.01, 1.5, step=0.01),
            "min_child_samples": trial.suggest_int("min_child_samples", 3, 299, step=2),
            "scale_pos_weight": trial.suggest_float("scale_pos_weight", 1.0, 3.0, step=0.01)
        }

        model = cat.CatBoostClassifier(boosting_type="Ordered", loss_function="Logloss", task_type="GPU",
                                    n_estimators=int(ntrees * 0.2),
                                    learning_rate=eta / 10,
                                    one_hot_max_size=3, leaf_estimation_method="Gradient",
                                    # leaf_estimation_iterations=5,
                                    # max_ctr_complexity=2,
                                    logging_level="Silent", random_state=fold, thread_count=cpu_count(),
                                    **tuning_params)
        model.fit(train_x, train_y, cat_features=categoIdx)
    elif model_name == "XGB_GBT":
        tuning_params = {
            # "n_estimators" : trial.suggest_int("n_estimators", 500, 5000, step=500),
            # "learning_rate" : trial.suggest_categorical("learning_rate", [1e-2, 5e-2, 1e-1]),
            "max_depth": trial.suggest_int("max_depth", 4, 8, step=1),
            "subsample": trial.suggest_float("subsample", 0.5, 0.95, step=0.05),
            "colsample_bynode": trial.suggest_float("colsample_bynode", 0.5, 0.95, step=0.05),
            "reg_lambda": trial.suggest_float("reg_lambda", 0.01, 10.0, step=0.01),
            # "min_child_weight": trial.suggest_float("min_child_weight", 0.01, 10.0, step=0.01),
            # "gamma": trial.suggest_float("gamma", 0.0, 10.0, step=0.01),
            # "max_delta_step":  trial.suggest_float("max_delta_step", 0.0, 10.0, step=0.01),
            "scale_pos_weight": trial.suggest_float("scale_pos_weight", 1.0, 3.0, step=0.01)
        }
        model = xgb.XGBClassifier(booster="gbtree", objective="binary:logistic",
                                tree_method="gpu_hist", gpu_id=0, sampling_method="gradient_based",
                                # n_estimators=int(ntrees * 0.2),
                                # learning_rate=eta / 10,
                                n_estimators=ntrees,
                                learning_rate=eta,
                                random_state=fold, verbosity=0, use_label_encoder=False,
                                **tuning_params)     
        # model.fit(train_x, train_y, verbose=False)
        model.fit(train_x, train_y, eval_metric=xgb_f1_score, verbose=False,
              eval_set=[(val_x, val_y)], early_stopping_rounds=int(ntrees * 0.2))
    # default randomforest
    else:
        tuning_params = {
            "num_leaves": trial.suggest_categorical("num_leaves", [pow(2, i) - 1 for i in [4, 5, 6, 7, 8]]),
            "subsample": trial.suggest_float("subsample", 0.5, 0.9, step=0.05),
            "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 0.9, step=0.05),
            "reg_lambda": trial.suggest_float("reg_lambda", 0.01, 10.0, step=0.01),
            "min_child_samples": trial.suggest_int("min_child_samples", 5, 50, step=5),
            "min_child_weight": trial.suggest_float("min_child_weight", 1e-4, 1e-2),
            "scale_pos_weight": trial.suggest_float("scale_pos_weight", 0.5, 2.0, step=0.01)
        }
        model = lgb.LGBMClassifier(boosting_type="rf", objective="binary", force_row_wise=True,
                                   n_estimators=int(ntrees * 0.2), subsample_freq=1, device_type="gpu", gpu_device_id=0,
                                   n_jobs=cpu_count(), random_state=fold, **tuning_params)
        model.fit(train_x, train_y, categorical_feature=categoIdx,
                  callbacks=[lgb.callback.log_evaluation(period=np.inf, show_stdv=False)])

    prob = model.predict_proba(val_x)
    pred = [1 if i > cut_off else 0 for i in prob[:, 1]]
    opt_score = metrics.f1_score(val_y, pred)

    trial.report(opt_score, step=trial.number)
    if trial.should_prune():
        raise optuna.exceptions.TrialPruned()

    return opt_score

# XGBoost

In [ ]:
# ===== modeling =====
ntrees = 5000
eta = 1e-2

def xgb_f1_score(predt: np.ndarray, dtrain: xgb.DMatrix):
    return 'F1', -metrics.f1_score(dtrain.get_label(), [1 if i > cut_off else 0 for i in predt])
model_name = "XGBoost_Gbtree_Try5"
total_time = time()
seed_everything()
for fold, (nonkIdx, kIdx) in enumerate(kfolds_spliter.split(full_x, full_y)):

    fold_time = time()
    print("===== Fold", fold+1, "Prediction =====\n")
    fold_dir = "fold_" + str(fold)
    wnb_init_info = {"project": "dacon_job_recommendation",
                     "group": model_name,
                     "name": fold_dir + "_optuna"}

    print("<XGBoost>")

    # optuna_cb = [optuna_wnbcallback(metric_name="f1_optuna",
    #                                 wandb_kwargs=wnb_init_info)]
    wandb.init(**wnb_init_info)
    wandb.config.step = 0

    train_ds_x = full_x_oh.iloc[nonkIdx].reset_index(drop=True).copy()
    train_ds_y = full_y.iloc[nonkIdx].reset_index(drop=True).copy()
    val_ds_x = full_x_oh.iloc[kIdx].reset_index(drop=True).copy()
    val_ds_y = full_y.iloc[kIdx].reset_index(drop=True).copy()
    test_ds_x = test_x_oh.copy()
    
    # optuna_timout = int(6 * 3600 / kfolds_spliter.get_n_splits())
    # # optuna_pruner = pruners.MedianPruner(n_warmup_steps=150, interval_steps=5, n_min_trials=20)
    # optuna_study = create_study(direction='maximize', sampler=TPESampler())
    # optuna_study.optimize(lambda trial: optuna_objective_function(
    #     trial, train_ds_x, train_ds_y, val_ds_x, val_ds_y, "XGB_GBT", ntrees, eta),
    #                       n_jobs=1, n_trials=300, timeout=optuna_timout, callbacks=optuna_cb)
    
    # optuna_plt.plot_optimization_history(optuna_study).show(renderer="browser")
    # optuna_plt.plot_param_importances(optuna_study).show(renderer="browser")

    model = xgb.XGBClassifier(booster="gbtree", objective="binary:logistic",
                            tree_method="gpu_hist", gpu_id=0, sampling_method="gradient_based",
                            n_estimators=ntrees,
                            learning_rate=eta,
                            max_depth=6, subsample=0.8, colsample_bynode=0.5, scale_pos_weight=1.5,
                            random_state=fold, verbosity=0, use_label_encoder=False)

    model.fit(train_ds_x, train_ds_y, eval_metric=xgb_f1_score, verbose=False,
              eval_set=[(val_ds_x, val_ds_y)], early_stopping_rounds=int(ntrees * 0.2))

    # model.fit(train_ds_x, train_ds_y, verbose=False)
    
    # print("tuned params --->", optuna_study.best_params)
    print("best trees --->", model.best_iteration, "\n")

    prob = model.predict_proba(val_ds_x)
    pred = [1 if i > cut_off else 0 for i in prob[:, 1]]
    val_perf.append(metrics.f1_score(val_ds_y, pred))
    val_prob[kIdx] = prob
    
    print("Fold running time --->", np.round(time() - fold_time, 3))
    print(model_name, "Fold", fold + 1, "F1 Score --->", val_perf[-1])
    print(model_name, "Average F1 Score --->", np.mean(val_perf), "\n\n")
    cm = wandb.plot.confusion_matrix(
        y_true=list(val_ds_y),
        preds=pred,
        class_names=[0, 1])
    wandb.log(data={"fold": fold, "f1_final": val_perf[-1], "ntrees": model.best_iteration, "conf_mat": cm}, commit=True)
    
    prob = model.predict_proba(test_ds_x)
    test_prob += prob / kfolds_spliter.get_n_splits()
    wandb.finish()

===== Fold 1 Prediction =====

<XGBoost>


tuned params ---> {'max_depth': 4, 'subsample': 0.8500000000000001, 'colsample_bynode': 0.8500000000000001, 'reg_lambda': 5.82, 'scale_pos_weight': 2.69}
best trees ---> 4836 

Fold running time ---> 697.106
XGBoost_Gbtree_Try5 Fold 1 F1 Score ---> 0.6915467768228797
XGBoost_Gbtree_Try5 Average F1 Score ---> 0.6915467768228797 




f1_final,▁
fold,▁
ntrees,▁
f1_final,0.69155
fold,0
ntrees,4836


===== Fold 2 Prediction =====

<XGBoost>


tuned params ---> {'max_depth': 4, 'subsample': 0.8500000000000001, 'colsample_bynode': 0.8500000000000001, 'reg_lambda': 5.82, 'scale_pos_weight': 2.69}
best trees ---> 4932 

Fold running time ---> 698.527
XGBoost_Gbtree_Try5 Fold 2 F1 Score ---> 0.6901759253420771
XGBoost_Gbtree_Try5 Average F1 Score ---> 0.6908613510824784 




...

tuned params ---> {'max_depth': 4, 'subsample': 0.8500000000000001, 'colsample_bynode': 0.8500000000000001, 'reg_lambda': 5.82, 'scale_pos_weight': 2.69}
best trees ---> 4980 

Fold running time ---> 688.565
XGBoost_Gbtree_Try5 Fold 9 F1 Score ---> 0.6899066925590868
XGBoost_Gbtree_Try5 Average F1 Score ---> 0.6897716644123743 




f1_final,▁
fold,▁
ntrees,▁
f1_final,0.68991
fold,8
ntrees,4980


===== Fold 10 Prediction =====

<XGBoost>


tuned params ---> {'max_depth': 4, 'subsample': 0.8500000000000001, 'colsample_bynode': 0.8500000000000001, 'reg_lambda': 5.82, 'scale_pos_weight': 2.69}
best trees ---> 4994 

Fold running time ---> 682.421
XGBoost_Gbtree_Try5 Fold 10 F1 Score ---> 0.6898507791941761
XGBoost_Gbtree_Try5 Average F1 Score ---> 0.6897795758905545 




f1_final,▁
fold,▁
ntrees,▁
f1_final,0.68985
fold,9
ntrees,4994


In [ ]:
wandb.finish()

# Submission

In [ ]:
print(model_name)
createFolder(folder_path + "models/" + model_name + "/")
dataframe(val_prob).to_csv(folder_path + "models/" + model_name + "/val_prob.csv", index=False)
dataframe(test_prob).to_csv(folder_path + "models/" + model_name + "/test_prob.csv", index=False)
submission = read_csv(folder_path + "sample_submission.csv")
submission[target_var] = [1 if i > cut_off else 0 for i in test_prob[:, 1]]
submission.to_csv(folder_path + "models/" + model_name + "/submission_" + model_name + ".csv", index=False)
submission.head(10)

XGBoost_Gbtree_Try5


,id,target
0,0,1
1,1,0
2,2,0
3,3,1
4,4,1
5,5,0
6,6,1
7,7,0
8,8,1
9,9,0
